# Pandas and Scikit-Learn

[Pandas](http://pandas.pydata.org) and [scikit-learn](http://scikit-learn.org/stable/) are two of the most popular scientific python libraries.
Pandas is commonly used to preprocess, reshape, and transform the data prior to handing it to scikit-learn to fit a model.

## Three-Minute Intro to Scikit-Learn

It's the goto library for machine learning in Python.
They use a consistent API for specifiying and fitting models.
For *supervised* learning tasks, you have a *feature matrix* `X`, that's an `[N x P]` NumPy array, and a *target array* `y`, that's typically a 1-dimensional array with length `N`. 

In [ ]:
from sklearn.datasets import load_boston

boston = load_boston()

y = boston['target']
X = boston['data']
print(boston['feature_names'])

In [ ]:
y[:5]

In [ ]:
X[:5]

Scikit-learn cleanly separates the *model specification* from the *model fitting*.

You specify your model by instantiating an *estimator*, for example `sklearn.linear_model.LinearRegression`.

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression(normalize=True)

You can set *hyperparameters* (parameters that are "outside", or not learned by the model) when you specify the model. `normalize=True` is a hyperparameter that tells scikit-learn to normalize the data before fitting.

Then you fit the model by passing the data (feature matrix `X` and target array `y`) to the `.fit` method.
At this point, the estimator *learns the parameters* that best fit the data.
For a linear regression, that's the `.coef_` attribute, which stores the parameters of the linear model (one per feature, plus an intercept by default).

In [ ]:
model.fit(X, y)

In [ ]:
model.coef_

## The Problem

1. Different data models:
    - NumPy is homogenous, n-dimensional arrays
    - Pandas is heterogenous, 2-dimensional tables
3. Pandas has additional dtypes

Pandas and scikit-learn have largely overlapping, but still different data models.
Scikit-learn uses NumPy arrays for most everything (the exceptiong being scipy sparse matricies for certain tasks, which we'll ignore).
Pandas builds on top of NumPy, but has made several extensions to its type system, creating a slight rift between the two. Most notably, pandas supports heterogenous data and has added several extension data-types on top of NumPy.

## 1. Homogeneity vs. Heterogeneity

NumPy `ndarray`s (and so scikit-learn feature matrices) are *homogeneous*, they must have a single dtype, regardless of the number of dimensions.
Pandas `DataFrame`s are potentially *heterogenous*, and can store columns of multiple dtypes within a single DataFrame.

In [ ]:
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
x = np.array([
    [10, 1.0],  # mix of integer and floats
    [20, 2.0],
    [30, 3.0],
    ])
x.dtype

In [ ]:
df = pd.DataFrame([
    [10, 1.0],
    [20, 2.0],
    [30, 3.0]
])
df.dtypes

## 2. Extension Types

Pandas has implemented some *extension dtypes*: `Categoricals` and datetimes with timezones.

These extension types cannot be expressed natively as NumPy arrays, *even if they are a single homogenouse dimension*, and must go through some kind of (potentially lossy) conversion process when converting to NumPy.

In [ ]:
s = pd.Series(pd.Categorical(['a', 'b', 'c', 'a'],
                             categories=['d', 'a', 'b', 'c'],
                             ordered=True))
s

Casting this to a NumPy array loses the categories and ordered information.

In [ ]:
np.asarray(s)

"Real-world" data is often complex and heterogeneous, making pandas the tool of choice.
However, tools like scikit-learn, which do not depend on pandas, can't use its
richer data model.

In my experience, most of the time the different data models aren't an issue.
Recent versions of scikit-learn are much better about taking and returning DataFrames where possible (e.g. `train_test_split`).
That said, there are a few rough edges that you can run into.
In these cases, we need a way of bridging the gap between pandas' DataFrames and the NumPy arrays appropriate for scikit-learn.
Fortunately the tools are all there to make this conversion smooth.

## The Data

For our example we'll work with a simple dataset on tips:

In [ ]:
df = pd.read_csv("data/tips.csv")
df.head()

In [ ]:
df.info()

<div class="alert alert-success" data-title="Target, Feature arrays">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Target, Feature arrays</h1>
</div>
<p>Split the DataFrame `df` into a `Series` called `y` containing the `tip` amount, and a DataFrame `X` containing everything else.

Our target variable is the tip amount. The remainder of the columns make up our features.</p>

In [ ]:
%load solutions/sklearn_pandas_split.py

In [ ]:
y.head()

In [ ]:
X.head()

Notice the feature matrix is a mixture of numeric and categorical variables.
In statistics, a categorical variable is a variable that comes from a limited, fixed set of values.
At the moment though, the actual data-type of those columns is just `object`, containing python strings. We'll convert those to pandas `Categorical`s later.

## The Stats

Our focus is about how to use pandas and scikit-learn together, not how to build the best tip-predicting model.
To keep things simple, we'll fit a linear regression to predict `tip`, rather than some more complicated model.

In [ ]:
from sklearn.linear_model import LinearRegression

When you fit a linear regression, you (or scikit-learn, rather) end up having to solve an equation to find the line that minimizes the mean squared error between the predictions and observations. The equation that gives the best-fit line is

$$
\hat{\boldsymbol{\beta}} = \left(\boldsymbol{X}^T\boldsymbol{X}\right)^{-1} \boldsymbol{X}^T \boldsymbol{y}
$$

where

- $\hat{\boldsymbol{\beta}}$ is our estimate for the vector of coefficients describing the best-fit line (`LinearRegression.coef_`)
- $\boldsymbol{X}$ is the feature matrix
- $\boldsymbol{y}$ is the target array (tip amount)

There's no need to worry about that equation; it likely won't make sense unless you've seen it before.
The only point I want to emphasize is that finding the best-fit line requires doing some matrix multiplications.
If we just tried to fit a regression on our raw data, we'd get an error:

In [ ]:
%xmode Plain
lm = LinearRegression()
lm.fit(X, y)

The message, "could not convert string to float" says it all.
We (or our library) need to somehow convert our *categorical* data (`sex`, `smoker`, `day`, and `time`) into numeric data.
The next two sections offer some possible ways of doing that conversion.

## Dummy Encoding

![dummy](figures/dummy.png)

Dummy encoding is one approach to converting categorical to numeric data.
It expands each categorical column to *multiple* columns, one per distinct value.
The values in these new dummy-encoded columns are either 1, indicating the presence of that value in that observation, or 0.
Versions of this are implemented in both scikit-learn and pandas.

I recommend the pandas version, `get_dummies`. It offers a few conveniences:

- Operates on multiple columns at once
- Passes through numeric columns unchanged
- Preserves row and column labels
- Provides a `drop_first` keyword for dropping a level per column. You might want this to avoid [perfect multicolinearity](https://en.wikipedia.org/wiki/Multicollinearity) if you have an intercept
- Uses Categorical information (more on this later)

In [ ]:
X_dummy = pd.get_dummies(X)
X_dummy.head()

In [ ]:
lm = LinearRegression()
lm.fit(X_dummy, y)

pd.Series(lm.coef_, index=X_dummy.columns)

## Refinements

Our last approach worked, but there's still room for improvement.

1. We can't easily go from dummies back to categoricals
2. Doesn't integrate with scikit-learn `Pipeline` objects.
3. If working with a larger dataset and `partial_fit`, codes could be missing from subsets of the data.
4. Memory inefficient if there are many records relative to distinct categories

These items become more important when you go to "productionize" your model.
But keep in mind that we've solved the basic problem of moving from pandas DataFrames to NumPy arrays for scikit-learn; now we're just making the bridge sturdier.

To accomplish this we'll store additonal information in the *type* of the column and write a [Transformer](http://scikit-learn.org/stable/modules/generated/sklearn.base.TransformerMixin.html) to handle the conversion to and from dummies.

## Aside: scikit-learn Pipelines

Rarely when doing data analysis do we take plug a raw dataset directly into a model.
There's typically some preprocessing and feature engineering before the fitting stage.
`scikit-learn` provides the `Pipeline` interface for chaining together a sequence of fit and transform steps. For example, suppose we wanted our pipeline to be

- standardize each column (subtract the mean, normalize the variance to 1)
- compute all the [interaction terms](https://en.wikipedia.org/wiki/Interaction_(statistics))
- fitting a Lasso regression

Without using a scikit-learn `Pipeline`, you need to assign the output of each step to a temporary variable, and manually shuttling data through to the end:

In [ ]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Lasso

In [ ]:
X_scaled = StandardScaler().fit_transform(X_dummy, y)
X_poly = (PolynomialFeatures(interaction_only=True)
            .fit_transform(X_scaled, y))
model = Lasso(alpha=.5)
model.fit(X_poly, y)

With pipelines, this becomes:

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
pipe = make_pipeline(StandardScaler(),
                     PolynomialFeatures(interaction_only=True),
                     Lasso(alpha=.5))
pipe.fit(X_dummy, y)

I always recommend the pipeline version.
For one thing, I prefer the aesthetics.
Especially with longer chains of computations, pipelines remove the need for many temporary variables.
They are also easier to save to disk with [joblib](https://pythonhosted.org/joblib/persistence.html).
But the most important reason is the interaction of `Pipeline` and [`GridSearchCV`](http://scikit-learn.org/stable/modules/grid_search.html).

When fitting a model you'll typically have a space of *hyperparameters* to search over.
These are the parameters passed to each estimators `__init__` method, so before the `.fit` step.
In the pipeline above, some examples of hyperparameters are the `interaction_only` parameter of `PolynomialFeatures` and the `alpha` parameter of `Lasso`.

A common mistake in machine learning is to let information from your test dataset leak into your training dataset by preprocessing *before* splitting.
This means the score you get on the test may not be an accurate representation of the score you'll get on new data.

`scikit-learn` provides many tools for you to write custom transformers that work well in its `Pipeline`.
When writing a custom transformer, you should:

- inherit from `sklearn.base.TransformerMixin`
- implement a `.fit` method that takes a feature matrix `X` and a target array `y`, returning `self`.
- implement a `.transform` method that also takes an `X` and a `y`, returning the transformed feature matrix

Below, we'll write a couple custom transformers to make our last regression more robust. But before that, we need to examine one of pandas' extension dtypes.

## Pandas `Categorical` dtype

We've already talked about Categoricals, but as a refresher:

- There are a fixed set of possible values the variable can take
- The cateogories can be ordered or unordered
- The array of data is dictionary encoded, so the set of possible values is stored once, and the array of actual values is stored efficiently as an array of integers

`Categorical`s can be constructed either with the `pd.Categorical` constructor, or using the `.astype` method on a `Series`. For example

In [ ]:
day = df['day'].astype('category').head()
day.head()

With `.astype('category')` we're just using the defaults of

- The set of categories is just the set present in the column
- There is no ordering

The categorical-specific information of a `Series` is stored under the `.cat` accessor.

In [ ]:
day.cat.categories

In [ ]:
day.cat.ordered

The following class is a transformer that transforms to categorical columns.

In [ ]:
from sklearn.base import TransformerMixin

class CategoricalTransformer(TransformerMixin):
    "Converts a set of columns in a DataFrame to categoricals"
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        'Records the categorical information'
        self.cat_map_ = {col: X[col].astype('category').cat
                         for col in self.columns}
        return self

    def transform(self, X, y=None):
        X = X.copy()
        for col in self.columns:
            X[col] = pd.Categorical(X[col],
                                    categories=self.cat_map_[col].categories,
                                    ordered=self.cat_map_[col].ordered)
        return X
    
    def inverse_transform(self, trn, y=None):
        trn = trn.copy()
        trn[self.columns] = trn[self.columns].apply(lambda x: x.astype(object))
        return trn

The most important rule when writing custom objects to be used in a `Pipeline` is that the `transfrom` and `inverse_transform` steps shouldn't modify `self`. That should only occur in `fit`. Because we inherited from `TransformerMixin`, we get the `fit_transform` method.

In [ ]:
ct = CategoricalTransformer(columns=['sex', 'smoker', 'day', 'time'])
X_cat = ct.fit_transform(X)
X_cat.info()

## DummyEncoder

We now have the pieces in place to solve all our issues.
We'll write a class `DummyEncoder` for use in a scikit-learn `Pipeline`.
The entirety is given in the next cell, but we'll break it apart piece by piece.

In [ ]:
class DummyEncoder(TransformerMixin):
    
    def fit(self, X, y=None):
        self.columns_ = X.columns
        self.cat_cols_ = X.select_dtypes(include=['category']).columns
        self.non_cat_cols_ = X.columns.drop(self.cat_cols_)
        self.cat_map_ = {col: X[col].cat for col in self.cat_cols_}
        
        self.cat_blocks_ = {}  # {cat col: slice}
        left = len(self.non_cat_cols_)
        for col in self.cat_cols_:
            right = left + len(self.cat_map_[col].categories)
            self.cat_blocks_[col] = slice(left, right)
            left = right
        return self
    
    def transform(self, X, y=None):
        return np.asarray(pd.get_dummies(X))
    
    def inverse_transform(self, trn, y=None):
        numeric = pd.DataFrame(trn[:, :len(self.non_cat_cols_)],
                               columns=self.non_cat_cols_)
        series = []
        for col, slice_ in self.cat_blocks_.items():
            codes = trn[:, slice_].argmax(1)
            cat = self.cat_map_[col]
            cat = pd.Categorical.from_codes(codes,
                                            cat.categories,
                                            cat.ordered)
            series.append(pd.Series(cat, name=col))
        return pd.concat([numeric] + series, axis='columns')[self.columns_]

self = DummyEncoder()
trn = self.fit_transform(X_cat)
trn

## `.transform`

The transform method is the simplest, it's using `pd.get_dummies` like we did before.
That is wrapped in a `np.asarray` to convert the DataFrame to a NumPy array, simulating what would happen if we pass the dummy-encoded class to a scikit-learn transformer that deals with NumPy arrays.

## `.fit`

In `.fit`, we need to store all the information needed to take the `trn` NumPy array and go back to a `DataFrame` in the `.inverse_transform` step. This includes

- Column names (`self.columns_`)
- Cateogrical information (`self.cat_map_`)
- Mapping original columns to transformed positions (`self.non_cat_cols_` and `self.cat_blocks_`)

## numeric

The first thing to realize is that pandas `get_dummies` returns the un-touched (numeric) columns first. We had two of those, `total_bill` and `size`, and collect those first in `inverse_transform`.

In [ ]:
numeric = pd.DataFrame(trn[:, :len(self.non_cat_cols_)],
                       columns=self.non_cat_cols_)
numeric.head()

While we had two columns in this dataset, in general we'll have `len(self.non_cat_cols_)`.

## categoricals

The rest of `inverse_transform` deals with `categoricals`.
We have two separate tasks here.

1. Know which of the expanded columns in `trn` belong to which original categorical columns
2. Know the categorical attributes (`ordered`, `categories`) for that categorical

For the first task, we use the information stored in `self.cat_blocks_`. This is a dictonary mapping categorical column names to `slice` objects, that can be used on `trn`.

In [ ]:
self.cat_blocks_

In [ ]:
trn[:, self.cat_blocks_['day']][:5]

Each of these "blocks" contain the dummy-encoded categorical. To invert the dummy-encoded, we use the fact that each row only has a single `1`, meaning that the maximum value of the row is the category for that row. We'll find the `argmax` for each row.

In [ ]:
codes = trn[:, self.cat_blocks_['day']].argmax(1)
codes[:5]

Now we'll use the alternative `Categorical.from_codes` constructor, which is appropriate when you have an array of `codes`, and already know the `categories` and `ordered`. We've stored that categorical information in the `self.cat_map_` dictionary.

In [ ]:
cat = pd.Categorical.from_codes(codes, self.cat_map_['day'].categories,
                                self.cat_map_['day'].ordered)
cat

The rest of `inverse_transform` collects all the series of `Categorical` and concats them together with the `numeric` columns.

## Using our pipeline

In [ ]:
columns = ['sex', 'smoker', 'day', 'time']
pipe = make_pipeline(CategoricalTransformer(columns), DummyEncoder(), LinearRegression())
pipe.fit(X, y)

In [ ]:
yhat = pipe.predict(X)
sns.jointplot(y, y-yhat)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pipe = make_pipeline(CategoricalTransformer(columns), DummyEncoder(), PCA())
trn = pipe.fit_transform(X)
sns.jointplot(trn[:, 0], trn[:, 1]);

In [ ]:
pipe.inverse_transform(trn).head()

## Summary

We explored some of the differences between the scikit-learn (NumPy) and pandas data models.
We needed to convert a heterogenous pandas `DataFrame` to a homogonous `ndarray` for use in scikit-learn.
Specifically we used `pd.get_dummies` to dummy encode the categorical data.
After dummy encoding, we sucessfully fit the model. 

For a more robust method, we implmented two scikit-learn `Pipeline` compatible transformers.
The first converted columns of strings into proper pandas `Categorical`s.
The second used `pd.get_dummies` to transform `Categoricals`, storing all the information needed to reverse the transformation.